In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import matplotlib.pyplot as plt
import os
from PIL import Image
from PIL import UnidentifiedImageError
import torch
from torchvision import datasets, transforms
import re
import pandas as pd
import random
import shutil
import datetime as dt
import torch.nn as nn
import torch
from pytorch_lightning import LightningModule

#LOADING TORCH MODELS
import torchvision.models as models
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# Shuffle images into validation and training data 
# Set the paths for the source folder and the destination folders for the training and validation sets
source_folder = r'/content/drive/MyDrive/Bilder'
train_folder = r'/content/drive/MyDrive/trainingset'
val_folder = r'/content/drive/MyDrive/validationset'
augmented_folder=r'/content/drive/MyDrive/Augmented_Image'
# Set the validation split percentage (e.g. 20% of the images will be used for validation)
val_split = 0.2

# Create the training and validation folders if they do not exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Get the list of all image files in the source folder
image_files = [filename for filename in os.listdir(source_folder) if filename.endswith('.jpg')]

augmented_files = [filename for filename in os.listdir(augmented_folder) if filename.endswith('.jpg')]

# Shuffle the image files randomly
random.shuffle(image_files)

# Split the image files into training and validation sets
val_size = int(len(image_files) * val_split)
train_files = image_files[val_size:] 
val_files = image_files[:val_size]
val_files_id =[re.search(r'ID_(\d+).*', x).group(1) for x in val_files]

# Copy the training images to the training set folder
for filename in train_files:
    src_path = os.path.join(source_folder, filename)
    dst_path = os.path.join(train_folder, filename)
    shutil.copy(src_path, dst_path)

for filename in augmented_files:
    match = re.search(r'ID_(\d+).*', filename)
    image_id = match.group(1)
    if(image_id in val_files_id):
        continue
    src_path = os.path.join(augmented_folder, filename)
    dst_path = os.path.join(train_folder, filename)
    shutil.copy(src_path, dst_path)


# Copy the validation images to the validation set folder
for filename in val_files:
    src_path = os.path.join(source_folder, filename)
    dst_path = os.path.join(val_folder, filename)
    shutil.copy(src_path, dst_path)

In [23]:
#loading datasets
excel_data = pd.read_excel(r'/content/drive/MyDrive/WD_Features_and_Costs.xlsx')
excel_data['Label'] = pd.cut(excel_data['Schadenhöhe'], [x for x in range(0,60001,10000)]+[90000],labels=[0,1,2,3,4,5,6])
excel_data['Label'] = excel_data['Label'].astype(int)

excel_data = excel_data[['ID', 'Material', 'Etage ganze Zahl', 'Baujahr','Dauer des Schadens in Tagen', 'Postleitzahl', 'Schadenhöhe','Label']]

Dummy_Variables=[pd.get_dummies(excel_data['Material'],prefix = 'Material', prefix_sep = ':')]
Dummy_Variables=pd.concat(Dummy_Variables,axis=1)
excel_data=pd.concat([excel_data,Dummy_Variables],axis=1)
excel_data['Material:Styropor'] = 0
excel_data["Age"]=int(dt.datetime.today().strftime("%Y"))-excel_data["Baujahr"]

excel_data = excel_data.drop(columns=["Material","Postleitzahl","Baujahr","Schadenhöhe"])

#Displaying first 4 details
excel_data[:4]

,ID,Etage ganze Zahl,Dauer des Schadens in Tagen,Label,Material:Beton,Material:Holz,Material:Stein,Material:Styropor,Age
0,1,3,4,4,1,0,0,0,56
1,2,0,10,2,0,0,1,0,22
2,3,1,5,0,1,0,0,0,107
3,4,3,7,6,1,0,0,0,59


## Custom Dataset

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.indices = list(range(len(data)))

    def __getitem__(self, index):
        # return both the data and the index
        return self.data[index], self.indices[index]

    def __len__(self):
        return len(self.data)

In [25]:
def formTensorData(directory, reshape_size):
    

    convert_tensor = transforms.ToTensor()

    #resize every image to input shape
    input_shape = (reshape_size, reshape_size)
    files = os.listdir(directory)
    inputImgs = []
    inputImgs_labels = []
    features = []
    for filename in files:
        
        match = re.search(r'ID_(\d+).*', filename)
        image_id = int(match.group(1))
        if filename.endswith('.jpg'):
            try:

                file_path = os.path.join(directory, filename)
                img = Image.open(file_path).convert('RGB')
                
                if os.path.getsize(file_path) > 0:
                    img = img.resize(input_shape[:2])
                    X = convert_tensor(img)
                    X_label = excel_data[excel_data['ID'] == image_id].iloc[0,:]['Label']

                    #preparing output
                    inputImgs_labels.append(X_label)
                    excel_features = excel_data[excel_data['ID'] == image_id].drop(columns=['Label','ID']).iloc[0,:]
                    inputImgs.append(X)
                    features.append(torch.Tensor(list(excel_features.values)))
                    
            except UnidentifiedImageError:
                print(f"{filename} is not a valid image file")
            
                
    return torch.stack(inputImgs), inputImgs_labels, features

In [26]:
X, X_label, features = formTensorData(train_folder,224)
X_valid, X_valid_label, val_features = formTensorData(val_folder,224)

In [27]:
X=X.to(DEVICE)
X_valid=X_valid.to(DEVICE)

In [28]:
X_label = torch.Tensor(X_label).type(torch.LongTensor).to(DEVICE)
features = torch.stack(features).to(DEVICE)
X_valid_label = torch.Tensor(X_valid_label).type(torch.LongTensor).to(DEVICE)
X_val_features = torch.stack(val_features).to(DEVICE)

## TRAIN & TEST : ADAM OPTIMIZER

In [31]:
# train model
from tqdm import tqdm

def m_train(
    model, num_epochs=5, learning_rate=0.001, batch_size=10, reshape_1d=False
):
    
    my_dataset = MyDataset(X)

    # create a DataLoader
    valid_loader = DataLoader(my_dataset, batch_size=batch_size)
    
    #Instantiating loss function
    loss_fn = nn.NLLLoss()
    
    #Instantiating optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for e in tqdm(range(num_epochs)):
        for batch, indices in valid_loader:
            inputs = batch
            labels = X_label[indices]
            batch_features = features[indices]
            forward = model(inputs,batch_features)
            loss = loss_fn(forward, labels)
            print(loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
    return model

In [32]:
# test model
@torch.no_grad()
def m_test(model, batch_size=5, reshape_1d=False):
    """
    Function to test your CNN on validation data

    Parameters
    ----------

    model: trained CNN from task 1a
    batch_size: size of batch for dataloader
    reshape_1d: Reshape images to a 1d vectors (allows use of models other than CNNs
                such as fully-connected FNNs)

    Returns
    -------
    accuracy of input model
    """
    correct = 0  # number of correct predictions
    total = 0  # total number of examples
    model.eval()  # set layers like dropout and batch norm to eval mode
    total_predictions = torch.tensor([]).to(DEVICE)

    # Create validation data loader
    
    my_dataset = MyDataset(X_valid)

    # create a DataLoader
    valid_loader = DataLoader(my_dataset, batch_size=batch_size)
    
    # Loop over data
    for batch,indices in valid_loader:
        indices=indices.to(DEVICE)
        inputs = batch
        labels = X_valid_label[indices]
        batch_features = X_val_features[indices]
        print(indices)
        print(labels)

        total += inputs.size(dim=0)
        
        out = model(inputs,batch_features)
        _, predictions = torch.max(out, dim = 1)
        print(predictions)
        
        total_predictions = torch.cat((total_predictions,predictions),dim=0)
        correct+= torch.sum(predictions == labels).item()
        pass

    accuracy = (correct / total) * 100
    print("Accuracy on {} validation images: {} %".format(total, accuracy))
    return accuracy,total_predictions

## SGD OPTIMIZER (less accuracy)

In [33]:
# # train model
# from tqdm import tqdm

# def mnist_train(
#     model, num_epochs=5, learning_rate=0.001, batch_size=10, reshape_1d=False
# ):
    
#     my_dataset = MyDataset(X)

#     # create a DataLoader
#     valid_loader = DataLoader(my_dataset, batch_size=batch_size)
    
#     #Instantiating loss function
#     loss_fn = nn.NLLLoss()
    
#     #Instantiating optimizer
#     # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     SGD_optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    
#     for e in tqdm(range(num_epochs)):
#         for batch, indices in valid_loader:
#             inputs = batch
#             labels = X_label[indices]
#             batch_features = features[indices]
#             forward = model(inputs,batch_features)
#             loss = loss_fn(forward, labels)
#             print(loss)
#             SGD_optimizer.zero_grad()
#             loss.backward()
#             SGD_optimizer.step()
        
#     return model

## CNN Pretrained

In [34]:
class CNN_Pretrained(nn.Module):
    def __init__(self):
        super(CNN_Pretrained, self).__init__()
        self.sequential = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),

            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),

            nn.Conv2d(16, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(),
        )

        self.backbone = models.resnet50(pretrained=True)
        
        #freeze pre-trained layer
        for param in self.backbone.parameters():
            param.requires_grad = False
                            
        self.linear1 = nn.Linear(1000, 249)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.5)
        self.linear2 = nn.Linear(249+7, 128)
        self.linear3 = nn.Linear(128,7)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, features):
        out = None
        out = self.sequential(x)
        out = self.backbone(out)
        out = self.linear1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = torch.cat((out,features),dim = 1)
        out = self.linear2(out)
        out = self.linear3(out)
        out = self.softmax(out)
        
        return out

In [35]:
# Train a model.
model_with_pretrain = m_train(CNN_Pretrained().to(DEVICE), batch_size=40, num_epochs = 10)
torch.save(model_with_pretrain, "/content/drive/MyDrive/cnn_pretrained.pt")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/10 [00:00<?, ?it/s]

tensor(2.8404, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3457, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7554, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9260, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8379, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6672, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6127, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7323, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6038, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5825, device='cuda:0', grad_fn=<NllLossBackward0>)


 10%|█         | 1/10 [00:04<00:43,  4.89s/it]

tensor(1.4435, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5142, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5753, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6050, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4032, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3842, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3817, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1608, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2820, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3769, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.0453, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2618, device='cuda:0', grad_fn=<NllLossBackward0>)


 20%|██        | 2/10 [00:10<00:40,  5.03s/it]

...

tensor(0.3196, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2222, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4211, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1446, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3548, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2139, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2606, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1677, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4498, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2335, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1099, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2246, device='cuda:0', grad_fn=<NllLossBackward0>)


 90%|█████████ | 9/10 [00:45<00:04,  4.96s/it]

tensor(0.5528, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3124, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4474, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3871, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3401, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4022, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2724, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4523, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2179, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2947, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3089, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 10/10 [00:49<00:00,  5.00s/it]

tensor(0.5015, device='cuda:0', grad_fn=<NllLossBackward0>)


In [36]:
accuracy,predictions = m_test(model_with_pretrain)

tensor([0, 1, 2, 3, 4], device='cuda:0')
tensor([6, 0, 1, 1, 4], device='cuda:0')
tensor([6, 0, 0, 5, 0], device='cuda:0')
tensor([5, 6, 7, 8, 9], device='cuda:0')
tensor([3, 4, 2, 1, 5], device='cuda:0')
tensor([3, 3, 2, 4, 1], device='cuda:0')
tensor([10, 11, 12, 13, 14], device='cuda:0')
tensor([2, 0, 1, 1, 0], device='cuda:0')
tensor([1, 3, 4, 4, 0], device='cuda:0')
tensor([15, 16, 17, 18, 19], device='cuda:0')
tensor([4, 4, 0, 5, 0], device='cuda:0')
tensor([1, 4, 3, 0, 1], device='cuda:0')
tensor([20, 21, 22, 23, 24], device='cuda:0')
tensor([2, 0, 1, 3, 1], device='cuda:0')
tensor([4, 3, 3, 1, 1], device='cuda:0')
tensor([25, 26, 27, 28], device='cuda:0')
tensor([4, 2, 6, 1], device='cuda:0')
tensor([4, 2, 3, 1], device='cuda:0')
Accuracy on 29 validation images: 34.48275862068966 %


# Concatenation UNet Images and CNN network

In [37]:
import torch
import pytorch_lightning as pl
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchmetrics
import numpy as np
import matplotlib.pyplot as plt
import os
import dill as pickle
import torchvision.transforms.functional as TF
import os
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms.functional as Func

class Block(pl.LightningModule):
    
    def _init_(self, in_channels, out_channels, kernel_size=3):
        super()._init_()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
        
    def forward(self, x):
        return self.conv(x)

class Encoder(pl.LightningModule):

    def _init_(self, in_channels, out_channels):
        super()._init_()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            Block(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Decoder(pl.LightningModule):

    def _init_(self, in_channels, out_channels):
        super()._init_()
        self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
        self.conv = Block(in_channels, out_channels)

    def forward(self, expand_input, contract_input):
        expand_output = self.up(expand_input)
        if expand_output.shape != contract_input.shape:
                expand_output = TF.resize(expand_output, size=contract_input.shape[2:])
        concat_output = torch.cat((expand_output, contract_input), dim=1)
        return self.conv(concat_output)

class UNet(pl.LightningModule):

    def _init_(self, chn=(3,64,128,256,512, 1024)):
        super()._init_()
        self.learning_rate = 1e-3 #1e-3

        self.train_pixel_acc = torchmetrics.classification.BinaryAccuracy(num_classes=2, ignore= 255, average='weighted')
        self.train_J = torchmetrics.classification.BinaryJaccardIndex(num_classes=2, ignore= 255, average='weighted')

        self.val_pixel_acc = torchmetrics.classification.BinaryAccuracy(num_classes=2, ignore= 255 , average='weighted')
        self.val_J = torchmetrics.classification.BinaryJaccardIndex(num_classes=2, ignore= 255 , average='weighted')

        self.test_pixel_acc = torchmetrics.classification.BinaryAccuracy(num_classes=2, ignore= 255, average='weighted')
        self.test_J = torchmetrics.classification.BinaryJaccardIndex(num_classes=2, ignore= 255, average='weighted')      
        # encoder
        self.first_layer = Block(chn[0], chn[1])
        self.encode_layers = nn.ModuleList()
        self.decode_layers = nn.ModuleList()
        
        for i in range(1, len(chn)-2):
            self.encode_layers.append(Encoder(chn[i],chn[i+1]))
            
        # bottleneck
        self.bottleneck = Block(chn[-2], chn[-1])
        # decoder
                    
        for i in range(len(chn)-1, 1, -1):
            self.decode_layers.append(Decoder(chn[i], chn[i-1]))
            
        self.final_layer = nn.Conv2d(chn[1], 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

        
        
        #############################

    def forward(self, x):
        # TODO: Define U-Net layers #
        x = self.first_layer(x)
        connections = []
        connections.append(x)
        for encode_layer in self.encode_layers:
            x = encode_layer(x)
            connections.append(x)
            
        x = self.bottleneck(x)
        connections = connections[::-1]

        for i in range(len(self.decode_layers)):
            x = self.decode_layers[i](x, connections[i])
                           
        return self.sigmoid(self.final_layer(x))
        #############################

    def decode_segmap(self, prediction):
        label_colors = torch.tensor([(0, 64, 128)])
        r = torch.zeros_like(prediction, dtype=torch.uint8)
        g = torch.zeros_like(prediction, dtype=torch.uint8)
        b = torch.zeros_like(prediction, dtype=torch.uint8)
        for l in range(0, self.num_class):
            idx = prediction == l
            r[idx] = label_colors[l, 0]
            g[idx] = label_colors[l, 1]
            b[idx] = label_colors[l, 2]
        rgb = torch.stack([r, g, b], axis=1)
        return rgb

    def training_step(self, train_batch, batch_idx):
        images, seg_mask = train_batch

        # convert the non-binary seg_mask to binary
        seg_mask_binary = torch.where(seg_mask > 0, torch.tensor(1), torch.tensor(0))
        seg_mask_binary = seg_mask_binary.squeeze(1).to(device)
        outputs = self(images)
        pred_seg_mask = torch.argmax(outputs, 1).to(device)
        outputs = outputs.squeeze(1).to(device)
        pred_seg_mask = pred_seg_mask.squeeze(1).to(device)
        # pixel-wise accuracy
        self.train_pixel_acc(pred_seg_mask, seg_mask_binary)

        # the Jaccard index (mean IoU)
        self.train_J(pred_seg_mask, seg_mask_binary)
        
        self.log('train_acc', self.train_pixel_acc, on_step=False, on_epoch=True)
        self.log('train_mIoU', self.train_J, on_step=False, on_epoch=True)

        # convert seg_mask_binary to float
        seg_mask_binary = seg_mask_binary.float()

        # loss
        loss = F.binary_cross_entropy(outputs.squeeze(1).to(device), seg_mask_binary.float())
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        print("Train :", loss, self.train_pixel_acc)
        return loss



    def validation_step(self, val_batch, batch_idx):
      images, seg_mask = val_batch

      seg_mask_binary = torch.where(seg_mask > 0, torch.tensor([1]).to(device), torch.tensor([0]).to(device))
      seg_mask_binary = seg_mask_binary.squeeze(1).to(device)
      outputs = self(images)
      pred_seg_mask = torch.argmax(outputs, 1)
      outputs = outputs.squeeze(1).to(device)
      pred_seg_mask = pred_seg_mask.squeeze(1).to(device)
      print(outputs.shape, pred_seg_mask.shape, seg_mask_binary.shape)
      # pixel-wise accuracy
      self.val_pixel_acc(pred_seg_mask, seg_mask_binary)

      # the Jaccard index (mean IoU)
      self.val_J(pred_seg_mask, seg_mask_binary)

      self.log('val_pixel_acc', self.val_pixel_acc, on_step=False, on_epoch=True)
      # self.log('val_mIoU', self.val_J, on_step=False, on_epoch=True)
      print("Valid:",  self.val_pixel_acc, self.val_J)
      loss = F.binary_cross_entropy(outputs.squeeze(1).to(device), seg_mask_binary.float())
      self.log('valid_loss', loss, on_step=False, on_epoch=True)
      return loss



    def test_step(self, batch, batch_idx):
      images, seg_mask = batch

      seg_mask_binary = torch.where(seg_mask > 0, torch.tensor([1]).to(device), torch.tensor([0]).to(device))
      seg_mask_binary = seg_mask_binary.squeeze(1).to(device)

      outputs = self(images)
      pred_seg_mask = torch.argmax(outputs, 1)
      outputs = outputs.squeeze(1).to(device)
      pred_seg_mask = pred_seg_mask.squeeze(1).to(device)
      print(outputs.shape, pred_seg_mask.shape, seg_mask_binary.shape)
      self.test_pixel_acc(pred_seg_mask, seg_mask_binary)
      self.test_J(pred_seg_mask, seg_mask_binary)
      print("Test:",  self.test_pixel_acc, self.val_J)
      self.log("test_acc", self.test_pixel_acc)
      # self.log("test_mIoU", self.test_J)

      loss = F.binary_cross_entropy(outputs.squeeze(1), seg_mask_binary.float())
      self.log('test_loss', loss, on_step=False, on_epoch=True)
      return loss
        

    def configure_optimizers(self):
        opt = torch.optim.SGD(self.parameters(), lr = self.learning_rate)
        return opt

In [38]:
# load the checkpoint
unet_model = torch.load('/content/drive/MyDrive/unet_water_damage.pt')
unet_model.eval()

UNet(
  (train_pixel_acc): BinaryAccuracy()
  (train_J): BinaryJaccardIndex()
  (val_pixel_acc): BinaryAccuracy()
  (val_J): BinaryJaccardIndex()
  (test_pixel_acc): BinaryAccuracy()
  (test_J): BinaryJaccardIndex()
  (first_layer): Block(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (encode_layers): ModuleList(
    (0): Encoder(
      (maxpool_conv): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): Block(
          (conv): Sequential(
            (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bi

In [39]:
class CNN_Pretrained_UNet(nn.Module):
    def __init__(self):
        super(CNN_Pretrained_UNet, self).__init__()
        self.unet = unet_model
         #freeze pre-trained layer
        for param in self.unet.parameters():
            param.requires_grad = False

        self.sequential = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),

            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),

            nn.Conv2d(16, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(),
        )
        self.unet_sequential = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(4, stride = 4),

            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU())

        self.backbone = models.resnet50(pretrained=True)
        
        #freeze pre-trained layer
        for param in self.backbone.parameters():
            param.requires_grad = False
                            
        self.linearUnet = nn.Linear(56*56*16,1000)
        self.linear1 = nn.Linear(2000, 249)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.5)
        self.linear2 = nn.Linear(249+7, 128)
        self.linear3 = nn.Linear(128,7)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, features):
        out = None
        out = self.sequential(x)
        out = self.backbone(out)
        out1 = self.unet(x)
        out1 = self.unet_sequential(out1)
        out1 = self.linearUnet(out1.view(-1,(56*56*16)))
        out = torch.cat((out,out1),dim = 1)
        out = self.linear1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = torch.cat((out,features),dim = 1)
        out = self.linear2(out)
        out = self.linear3(out)
        out = self.softmax(out)
        
        return out

In [40]:
# Train a model.
model_with_pretrain_unet = m_train(CNN_Pretrained_UNet().to(DEVICE), batch_size=40, num_epochs = 10)
torch.save(model_with_pretrain_unet, "/content/drive/MyDrive/cnn_pretrained_unet.pt")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor(3.0106, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(13.3048, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(12.1557, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(11.0361, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(7.9152, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.5518, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.9929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.1859, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6751, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.8035, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.8435, device='cuda:0', grad_fn=<NllLossBackward0>)


 10%|█         | 1/10 [00:12<01:55, 12.84s/it]

tensor(3.7481, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3154, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1381, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1852, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.4541, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.7782, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.0006, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9804, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8312, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8746, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8185, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6518, device='cuda:0', grad_fn=<NllLossBackward0>)


 20%|██        | 2/10 [00:26<01:46, 13.26s/it]

...

tensor(1.6042, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4867, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5707, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5660, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4796, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5072, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5175, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4532, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4975, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6198, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3422, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3945, device='cuda:0', grad_fn=<NllLossBackward0>)


 90%|█████████ | 9/10 [01:57<00:13, 13.03s/it]

tensor(1.5449, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4207, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4233, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3827, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4105, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3992, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4141, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4274, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4004, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4478, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4150, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5002, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 10/10 [02:10<00:00, 13.08s/it]

tensor(1.5664, device='cuda:0', grad_fn=<NllLossBackward0>)


In [41]:
accuracy,predictions = m_test(model_with_pretrain_unet)

tensor([0, 1, 2, 3, 4], device='cuda:0')
tensor([6, 0, 1, 1, 4], device='cuda:0')
tensor([1, 1, 3, 1, 3], device='cuda:0')
tensor([5, 6, 7, 8, 9], device='cuda:0')
tensor([3, 4, 2, 1, 5], device='cuda:0')
tensor([3, 4, 2, 2, 1], device='cuda:0')
tensor([10, 11, 12, 13, 14], device='cuda:0')
tensor([2, 0, 1, 1, 0], device='cuda:0')
tensor([2, 3, 4, 1, 1], device='cuda:0')
tensor([15, 16, 17, 18, 19], device='cuda:0')
tensor([4, 4, 0, 5, 0], device='cuda:0')
tensor([4, 2, 0, 1, 3], device='cuda:0')
tensor([20, 21, 22, 23, 24], device='cuda:0')
tensor([2, 0, 1, 3, 1], device='cuda:0')
tensor([1, 3, 3, 4, 1], device='cuda:0')
tensor([25, 26, 27, 28], device='cuda:0')
tensor([4, 2, 6, 1], device='cuda:0')
tensor([4, 0, 3, 4], device='cuda:0')
Accuracy on 29 validation images: 34.48275862068966 %
